In [11]:
import yaml
import os
from pathlib import Path
import pandas as pd

## Setup

In [12]:
# experiment setup
my_pdk = 'sky130'
my_design = 'add'
my_inst = '/add/adder0'
my_clock_period = 10 # ns

# useful paths
e2e_dpath = Path(os.getcwd()).parent
tests_dpath = e2e_dpath/f'experiments/tests-{my_pdk}'


## Generate experiment files

### Input files

In [13]:

N_ITER = 50

max8 = (1 << 8) - 1

max32 = (1 << 32) - 1
maxout32_in0 = int(max32/2)
maxout32_in1 = max32 - maxout32_in0

# tests dict
#   name format: <design_instance>-<test_name>
#       inputs: list, where each item is a list of items per line in input.txt
#       defines: for verilog
tests_dict = {
    'add8-zero': {
        'inputs': [(0,0) for _ in range(N_ITER)],
        'defines': dict(WIDTH=8),
    },
    'add8-max_input_switching': {
        'inputs': [((0,0) if i%2 else (max8,max8)) for i in range(N_ITER)],
        'defines': dict(WIDTH=8),
    },
    'add8-max_output_switching': {
        'inputs': [((0,0) if i%2 else (127,128)) for i in range(N_ITER)],
        'defines': dict(WIDTH=8),
    },
    'add32-zero': {
        'inputs': [(0,0) for _ in range(N_ITER)],
        'defines': dict(WIDTH=32),
    },
    'add32-max_input_switching': {
        'inputs': [((0,0) if i%2 else (max32,max32)) for i in range(N_ITER)],
        'defines': dict(WIDTH=32),
    },
    'add32-max_output_switching': {
        'inputs': [((0,0) if i%2 else (maxout32_in0,maxout32_in1)) for i in range(N_ITER)],
        'defines': dict(WIDTH=32),
    },
}

In [14]:
# create dirs
for t in tests_dict:
    # directory for all input/output files
    root = tests_dpath/t
    root.mkdir(exist_ok=True,parents=True)
    tests_dict[t]['defines']['TESTROOT'] = root
    tests_dict[t]['root'] = root
    # hammer build directory
    design = t.split('-')[0]
    obj_dir = f"build-{my_pdk}-cm/{design}"
    tests_dict[t]['design'] = design
    tests_dict[t]['obj_dir'] = obj_dir
    tests_dict[t]['obj_dpath'] = e2e_dpath/obj_dir
    

# write out input.txt
for t in tests_dict:
    with (tests_dict[t]['root']/'input.txt').open('w') as f:
        for i in tests_dict[t]['inputs']:
            f.write(" ".join(['{0:b}'.format(ii) for ii in i]) + '\n')


### Hammer Config

In [15]:
def write_cfg_rtl(test_dict):
  defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in test_dict['defines'].items() ] )
  cfg = f"""\
design.defines: &DEFINES
  - CLOCK_PERIOD=10
{defines_str}

sim.inputs:
  defines: *DEFINES
  defines_meta: 'append'

synthesis.inputs.defines: *DEFINES

power.inputs.defines: *DEFINES

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  input_files: [src/{my_design}.v]
  report_configs:
    - waveform_path: {test_dict['root']}/output.fsdb
      report_stem: {test_dict['root']}/power
      toggle_signal: clock
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
"""
  with (test_dict['root']/'config.yml').open('w') as f:
    f.write(cfg)

def write_cfg(test_dict):
  defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in test_dict['defines'].items() ] )
  cfg = f"""\
design.defines: &DEFINES
  - CLOCK_PERIOD=10
{defines_str}

sim.inputs.defines: *DEFINES
sim.inputs.defines_meta: 'append'

synthesis.inputs.defines: *DEFINES
power.inputs.defines: *DEFINES

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: syn
  report_configs:
    - waveform_path: {test_dict['root']}/output.fsdb
      report_stem: {test_dict['root']}/power
      toggle_signal: clock
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
"""
  with (test_dict['root']/'config.yml').open('w') as f:
    f.write(cfg)

for t in tests_dict:
    write_cfg(tests_dict[t])

## Run experiments

In [16]:
# generate custom make str for each test
make = f""
for t in tests_dict:
    cfg = str(tests_dict[t]['root']/'config.yml')
    tests_dict[t]['make'] = f"design={my_design} OBJ_DIR={tests_dict[t]['obj_dir']} extra={cfg}"

# build
build_dirs = {tests_dict[t]['obj_dir']: t for t in tests_dict} # run build once per build dir (not once per test)
for bd,t in build_dirs.items():
    print(f"make build -B {tests_dict[t]['make']} -B")
print()

# sim-rtl
for t in tests_dict:
    print(f"make redo-sim-rtl {tests_dict[t]['make']}")
print()

# power-rtl
for t in tests_dict:
    # re-use pre_report_power database if it's already generated (i.e. skip synthesis)
    make_target = "redo-power-rtl args='--only_step report_power'" \
            if (tests_dict[t]['obj_dpath']/'power-rtl-rundir/pre_report_power').exists() else 'power-rtl'
    print(f"make {make_target} {tests_dict[t]['make']}")
print()


make build -B design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add8-max_output_switching/config.yml -B
make build -B design=add OBJ_DIR=build-sky130-cm/add32 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add32-max_output_switching/config.yml -B

make redo-sim-rtl design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add8-zero/config.yml
make redo-sim-rtl design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add8-max_input_switching/config.yml
make redo-sim-rtl design=add OBJ_DIR=build-sky130-cm/add8 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add8-max_output_switching/config.yml
make redo-sim-rtl design=add OBJ_DIR=build-sky130-cm/add32 extra=/bwrcq/scratch/nayiri/hammer-sep24/e2e/experiments/tests-sky130/add32-zero/config.yml
make redo-sim-rtl design=add OBJ_DIR=bui

## Parse results

In [18]:

def parse_hier_power_rpt(fpath,inst) -> list:
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if inst == words[-1]:
            return [float(p) for p in words[2:6]]
    return []

power = list([])
for t in tests_dict:
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    power.append(parse_hier_power_rpt(fpath,my_inst))

power = pd.DataFrame(power,   #  mW
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys()) # type: ignore

energy = power * my_clock_period # mW * ns = pJ
energy

,Leakage,Internal,Switching,Total
add8-zero,0.000002,0.000000,0.0000,0.000002
add8-max_input_switching,0.000002,0.610563,11.3899,12.000500
add8-max_output_switching,0.000001,0.540167,12.8806,13.420800
add32-zero,0.000008,0.000000,0.0000,0.000008
add32-max_input_switching,0.000008,2.725520,50.6432,53.368800
add32-max_output_switching,0.000007,2.265580,51.5759,53.841500


## Notes
What do we want to test?

Inputs:
* 0 -> 0
* 0 -> 11...11
* different activity factors of adds

Designs:
* minimum critical path - can we force the synthesis tool to use faster gates

Flow:
* sim-rtl > power-rtl